# TO DO
1. Figure out dataset
2. Break down into individual characters
3. Form corpus from dataset for input and output characters
4. Assign a number to each character
5. Model will have an embedding so it will handle it
6. Output will be a bunch of integers so we will have to decode it

In [ ]:
!pip install wandb
# !pip install tensorflow
# !pip install keras

     |████████████████████████████████| 2.1MB 6.2MB/s 
     |████████████████████████████████| 133kB 21.2MB/s 
     |████████████████████████████████| 163kB 20.5MB/s 
     |████████████████████████████████| 102kB 6.8MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=0db091a4323c65ca1de304c7b11cba9318dae05237b8d07d829e3b26f6a33b61
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=37adf774a423f925fd97464e14d422680973e15f201ab2a4b27b3573687bb9cc
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [ ]:
import tarfile
import os
import pandas as pd
import keras
import numpy as np
import wandb
import tensorflow as tf
import shutil
from keras import layers
from keras.layers import LSTM, Dense, Embedding, Input
from keras.models import Model
from keras.utils.vis_utils import plot_model
from tqdm.auto import tqdm
from keras.layers import Lambda
from keras import backend as K

In [ ]:
!wget -nc https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

if not os.path.isdir('/content/dakshina_dataset_v1.0'):
  tarfile.open("/content/dakshina_dataset_v1.0.tar").extractall()

--2021-05-01 16:56:23--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.23.128, 74.125.203.128, 74.125.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.23.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   120MB/s    in 26s     

2021-05-01 16:56:49 (75.0 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
wandb.login(key='14394907543f59ea21931529e34b4d80d2ca8c9c')

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Loading Data

In [ ]:
class data_loader():

  @staticmethod
  def _load_raw_df(languages = ["hi", "mr"]):
    lex = dict()
    lex['train'], lex['val'], lex['test'] = [], [], [] 
    column_names = ['input', 'output', 'count']
    
    for la in languages:
      lex['train'].append(pd.read_csv('/content/dakshina_dataset_v1.0/'+la+'/lexicons/'+la+'.translit.sampled.train.tsv', sep='\t', header=None, names=column_names))
      lex['val'].append(pd.read_csv('/content/dakshina_dataset_v1.0/'+la+'/lexicons/'+la+'.translit.sampled.dev.tsv', sep='\t', header=None, names=column_names))
      lex['test'].append(pd.read_csv('/content/dakshina_dataset_v1.0/'+la+'/lexicons/'+la+'.translit.sampled.test.tsv', sep='\t', header=None, names=column_names))

    lex['train'] = pd.concat(lex['train'])
    lex['val'] = pd.concat(lex['val'])
    lex['test'] = pd.concat(lex['test'])

    return lex    

  @staticmethod
  def _make_final_df(lex):
    
    for div in ['train', 'val', 'test']:
    
      # removing non max transliterations
      idx = lex[div].groupby(['input'])['count'].transform(max) == lex[div]['count']
      lex[div] = lex[div][idx].reset_index(drop=True)

      # calclulating difference in lengths of various transliterations
      lex[div]['input_len'] = lex[div].apply(lambda x: len(str(x['input'])), axis=1)
      lex[div]['output_len'] = lex[div].apply(lambda y: len(str(y['output'])), axis=1)
      lex[div]['mod_dif'] = lex[div].apply(lambda z: abs(z['input_len'] - z['output_len']), axis=1) 

      # removing transliterations that vary by a lot in length
      idx = lex[div].groupby(['input'])['mod_dif'].transform(min) == lex[div]['mod_dif']
      lex[div] = lex[div][idx].reset_index(drop=True)

      # removing duplicates if any remain
      lex[div].drop_duplicates(subset='input', keep='first', inplace=True)

      # removing redundant columns
      lex[div].drop(labels=['count', 'input_len', 'output_len', 'mod_dif'], inplace=True, axis=1)

    return lex

  @staticmethod
  def _generate_batch(X, y, data_dict, num_decoder_tokens, batch_size = 1):
    ''' Generate a batch of data '''
    assert len(X)==len(y)
    ind = np.random.permutation(len(X))
    X,y = [X[j] for j in ind],[y[j] for j in ind]
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, data_dict['max_source_length']),dtype='float32')
            decoder_input_data = np.zeros((batch_size, data_dict['max_target_length']),dtype='float32')
            decoder_target_data = np.zeros((batch_size, data_dict['max_target_length'], num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text):
                  encoder_input_data[i, t] = word
                for t, word in enumerate(target_text):
                    if t<len(target_text)-1:
                        decoder_input_data[i, t] = word # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, word] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

  @staticmethod
  def _generate_batch_greedy(X, y, data_dict, num_decoder_tokens, batch_size = 1):
    ''' Generate a batch of data '''
    assert len(X)==len(y)
    ind = np.random.permutation(len(X))
    X,y = [X[j] for j in ind],[y[j] for j in ind]
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, data_dict['max_source_length']),dtype='float32')
            decoder_input_data = np.zeros((batch_size, 1),dtype='float32')
            # decoder_target_data = np.zeros((batch_size, data_dict['max_target_length'], num_decoder_tokens),dtype='float32')
            decoder_target_data = np.zeros((batch_size, 23, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text):
                  encoder_input_data[i, t] = word
                for t, word in enumerate(target_text):
                    if t==0 :
                        decoder_input_data[i, t] = 1 # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, word] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
class Tokenizer:

  def __init__(self, df):

    self.start_token = '<STR>'
    self.stop_token = '<STP>'
    self.unknown_token = '<UNK>'

    self.input_corpus = [self.start_token, self.stop_token, self.unknown_token]
    self.output_corpus = [self.start_token, self.stop_token, self.unknown_token]

    input_words = df.input.tolist()
    output_words = df.output.tolist()

    for word in input_words:
      tokens = str(word)
      for token in tokens:
        if token not in self.input_corpus:
          self.input_corpus.append(token)

    for word in output_words:
      tokens = str(word)
      for token in tokens:
        if token not in self.output_corpus:
          self.output_corpus.append(token)
    
    self.encode_dict_input = {self.input_corpus[i] : i+1 for i in range(len(self.input_corpus))}
    self.decode_dict_input = {k:v for v,k in self.encode_dict_input.items()}
    
    
    self.encode_dict_output = {self.output_corpus[i] : i+1 for i in range(len(self.output_corpus))}
    self.decode_dict_output = {k:v for v,k in self.encode_dict_output.items()}
    self.decode_dict_output.update({2:''})

  # takes in lists of words and returns lists of integers
  def encode(self, X, mode='input'):

    if (mode=='input'):
      input_list = []
      for word in X:
        word = str(word)
        # integer_list = [self.encode_dict_input[self.start_token]] + [self.encode_dict_input.get(token, self.encode_dict_input[self.unknown_token]) for token in word] + [self.encode_dict_input[self.stop_token]]
        integer_list =np.array([self.encode_dict_input.get(token, self.encode_dict_input[self.unknown_token]) for token in word])
        input_list.append(integer_list)
      
      return input_list
    
    if (mode=='output'):
      output_list = []
      for word in X:
        word = str(word)
        integer_list = np.array([self.encode_dict_output[self.start_token]] + [self.encode_dict_output.get(token, self.encode_dict_output[self.unknown_token]) for token in word] + [self.encode_dict_output[self.stop_token]])
        output_list.append(integer_list)
      
      return output_list
    

  # takes in lists of integers and returns lists of words
  def decode(self, X, mode='input'):

    if (mode=='input'):
      input_list = []
      for integers in X:
        token_list = [self.decode_dict_input.get(integer, '') for integer in integers] 
        input_list.append(''.join(token_list))
      
      return input_list

    if (mode=='output'):
      output_list = []
      for integers in X:
        token_list = [self.decode_dict_output.get(integer, '') for integer in integers[1:-1]] 
        output_list.append(''.join(token_list))
      
      return output_list

In [ ]:
def return_data_dict(languages=['hi', 'mr'], batch_size=32):

  lex = data_loader._load_raw_df(languages)
  lex = data_loader._make_final_df(lex)

  data_dict = dict()

  df_train = lex['train']
  df_val = lex['val']
  df_test = lex['test']

  tk = Tokenizer(df_train)

  data_dict['in_size'] = len(tk.input_corpus) + 1
  data_dict['out_size'] = len(tk.output_corpus) + 1

  X_train = tk.encode(df_train.input.tolist(), mode='input')
  Y_train = tk.encode(df_train.output.tolist(), mode='output')
  X_val = tk.encode(df_val.input.tolist(), mode='input')
  Y_val = tk.encode(df_val.output.tolist(), mode='output')
  X_test = tk.encode(df_test.input.tolist(), mode='input')
  Y_test = tk.encode(df_test.output.tolist(), mode='output')

  max_source_length = np.max(np.array([len(x) for x in X_train]))
  max_target_length = np.max(np.array([len(x) for x in Y_train]))

  data_dict['train'], data_dict['val'], data_dict['test']= dict(), dict(), dict()

  data_dict['train']['df'] = df_train
  data_dict['val']['df'] = df_val
  data_dict['test']['df'] = df_test

  

  data_dict['train']['max_source_length'] = np.max(np.array([len(x) for x in X_train]))
  data_dict['train']['max_target_length'] = np.max(np.array([len(x) for x in Y_train]))
  data_dict['val']['max_source_length'] = np.max(np.array([len(x) for x in X_val]))
  data_dict['val']['max_target_length'] = np.max(np.array([len(x) for x in Y_test]))
  data_dict['test']['max_source_length'] = np.max(np.array([len(x) for x in X_test]))
  data_dict['test']['max_target_length'] = np.max(np.array([len(x) for x in Y_test]))

  data_dict['max_source_length'] = max(data_dict['train']['max_source_length'],data_dict['val']['max_source_length'])
  data_dict['max_target_length'] = max(data_dict['train']['max_target_length'],data_dict['val']['max_target_length'])

  data_dict['train']['batch'] = data_loader._generate_batch(X_train, Y_train, data_dict['train'], data_dict['out_size'], batch_size)
  data_dict['train']['batch_greedy'] = data_loader._generate_batch_greedy(X_train, Y_train, data_dict['train'], data_dict['out_size'], batch_size)
  data_dict['val']['batch_greedy'] = data_loader._generate_batch_greedy(X_val, Y_val, data_dict['val'], data_dict['out_size'], batch_size)
  data_dict['val']['batch'] = data_loader._generate_batch(X_val, Y_val, data_dict['val'], data_dict['out_size'], batch_size)
  data_dict['test']['batch'] = data_loader._generate_batch(X_test, Y_test, data_dict['test'], data_dict['out_size'], batch_size)  
  data_dict['test']['batch_greedy'] = data_loader._generate_batch_greedy(X_test, Y_test, data_dict['test'], data_dict['out_size'], batch_size)    
  data_dict['tokenizer'] = tk

  return data_dict

In [ ]:
data_dict = return_data_dict()

In [ ]:
data_dict

In [ ]:
# data = dict()
# data['train'] = dict()
# data['train']['X'] = X_train
# data['train']['Y'] = Y_train
# data['in_size'] = len(tk.input_corpus) + 1
# data['out_size'] = len(tk.output_corpus) + 1
# num_decoder_tokens = data['out_size'] 
# max_source_length = np.max(np.array([len(x) for x in X_train]))
# max_target_length = np.max(np.array([len(x) for x in Y_train]))

# Question 1


In [ ]:
class BeamSearchCallBack(keras.callbacks.Callback):
  def __init__(self,details,test_data,tokenizer,out_size,num_val=200,beam=3) :
    self.details = details
    self.test_data = test_data
    self.tokenizer = tokenizer
    self.out_size = out_size
    self.num_val = num_val
    self.beam = beam

  def on_epoch_end(self,epoch,logs=None) :
    
    encoder_model = Model(self.details['encoder_inputs'], self.details['encoder_states'])
    rep_size = self.details['params']['rep_size']
    decoder_state_input = []
    for i in range(len(self.details['encoder_states'])) :
        new_state = Input(shape=(rep_size,))
        decoder_state_input.append(new_state)
    decoder_inputs = self.details['decoder_inputs']
    x = self.details['decoder_embedding'](decoder_inputs)
    
    for layer in self.details['decoder_layers'] :
      x, *decoder_states = layer(x,initial_state=decoder_state_input)

    x = self.details['decoder_dense'](x)
    decoder_model = Model(
        [decoder_inputs] + decoder_state_input,
        [x] + decoder_states )
    inp = self.tokenizer.encode(self.test_data['df'].input.tolist())
    out = self.tokenizer.encode(self.test_data['df'].output.tolist(),mode='output')
    out_size = self.out_size
    val_gen = data_loader._generate_batch(inp,out,self.test_data,self.out_size)
    acc = 0
    for i in tqdm(range(self.num_val)) :
      (input_seq,ans) , _ = next(val_gen)
      _,best = decode_sequence_beam(input_seq,self.beam,encoder_model,decoder_model,self.tokenizer,self.test_data['max_target_length'])
      w1 = self.tokenizer.decode(best,mode='output')
      w2 = self.tokenizer.decode(ans,mode='output')
      comp = (w1==w2)
      if comp :
        acc += 1    

    acc /= len(inp)
    print("Val Accuracy : "+str(acc))
    

In [ ]:
def decode_sequence_beam(input_seq,k,encoder_model,decoder_model,tk,max_target_length=20,getall=False):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq,batch_size=1,use_multiprocessing=True)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = 1 
    run_condition = [True for i in range(k)]
    # print(len(states_value))
    # print([target_seq] + [states_value])
    results, *states_values_temp = decoder_model.predict([target_seq] + [states_value])
    output_tokens = results

    states_values_k = [states_values_temp for i in range(k)]
    #get topk indices
    ind = np.argpartition(np.array(output_tokens[0, -1, :]), -k)[-k:]
    bestk_ind = ind
    output_tokens = np.array(output_tokens[0, -1, :])
    bestk_prob = output_tokens[ind]
    bestk_tot = [[1,bestk_ind[i]] for i in range(k)]
    # print(bestk_tot)

    
    while any(run_condition):
        bestk_tot_new = []
        bestk_prob_new = []
        states_values_k_new = []
        for i in range(k) :
            if run_condition[i] :
                a = bestk_tot[i]
                b = bestk_prob[i]
                target_seq[0,0] = a[-1]
                results,*states_values_temp = decoder_model.predict([target_seq] + states_values_k[i],batch_size=1)
                output_tokens = results

                states_values_k_temp = [states_values_temp for m in range(k)]

                states_values_k_new += states_values_k_temp
                ind = np.argpartition(np.array(output_tokens[0, -1, :]), -k)[-k:]
                bestk_ind = ind
                output_tokens = np.array(output_tokens[0, -1, :])
                bestk_prob_temp = output_tokens[ind]
                bestk_tot_temp = [a+[bestk_ind[j]] for j in range(k)]
                bestk_prob_temp2 = [b*bestk_prob_temp[j] for j in range(k)]
                bestk_prob_new += bestk_prob_temp2
                bestk_tot_new += bestk_tot_temp
            
            else :
                a = bestk_tot[i]
                b = bestk_prob[i]
                bestk_tot_new += [bestk_tot[i]]
                bestk_prob_new += [b]
                states_values_k_new += [states_values_k[i]]

        bestk_prob_new = np.array(bestk_prob_new)
        # print(len(bestk_prob_new),len(bestk_tot_new),len(states_values_k_new))
        ind = np.argpartition(bestk_prob_new,-k)[-k:]
        bestk_tot = [bestk_tot_new[i] for i in ind]
        states_values_k = [states_values_k_new[i] for i in ind]
        bestk_prob = bestk_prob_new[ind]
        run_condition = []
        for i in range(k) :
            a = bestk_tot[i]
            b = bestk_prob[i]
            if a[-1]!= 2 and len(a)<=max_target_length :
              run_condition.append(True)
            else :
              run_condition.append(False)

        # print(bestk_tot)

    final_words = []
    best_word = []
    best = 0.0
    for i in range(k) :
      a = bestk_tot[i]
      b = bestk_prob[i]
      final_words += [a]
      if b > best :
        best_word = [a]

    if getall :
      return (tk.decode(final_words,'output'),best_word)
    else :
      return final_words,best_word

In [ ]:
class rnn():

  def __init__(self, params):
    
    num_encode_layers = params['num_encode_layers']
    num_decode_layers = params['num_decode_layers']
    data_dict = params['data_dict']
    in_size = params['data_dict']['in_size']
    out_size = params['data_dict']['out_size']
    cell_type = params['cell_type']
    dropout = params['dropout']
    embed_size = params['embed_size']
    rep_size = params['rep_size']
        
    ###################### ENCODER NETWORK ######################
    
    encoder_inputs = Input(shape=(None,))
    x = Embedding(in_size, embed_size ,mask_zero=True)(encoder_inputs)

    encoder_layers = []
    
    for j in range(num_encode_layers-1) :   
      curr_layer = getattr(layers, cell_type)(rep_size, dropout=dropout, return_sequences=True)
      encoder_layers.append(curr_layer)
      x = curr_layer(x)

    curr_layer = getattr(layers, cell_type)(rep_size, dropout=dropout, return_state=True)
    encoder_layers.append(curr_layer)
    x, *encoder_states = curr_layer(x)

    ###################### DECODER NETWORK ######################

    decoder_inputs = Input(shape=(None,))

    decoder_embedding =  Embedding(out_size, embed_size, mask_zero=True)
    x = decoder_embedding(decoder_inputs)

    decoder_layers = []    
    
    for j in range(num_decode_layers) :
      curr_layer = getattr(layers, cell_type)(rep_size,dropout=dropout,return_state=True, return_sequences=True)
      decoder_layers.append(curr_layer)
      x, *decoder_states = curr_layer(x, initial_state=encoder_states)

    decoder_dense = Dense(units=out_size, activation='softmax')
    decoder_outputs = decoder_dense(x)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    self.model = model
    self.encoder_inputs = encoder_inputs
    self.encoder_layers = encoder_layers
    self.decoder_inputs = decoder_inputs
    self.decoder_embedding = decoder_embedding
    self.decoder_layers = decoder_layers
    self.decoder_dense = decoder_dense
    self.encoder_states = encoder_states
    self.params = params
    self.details = {
        'model' : self.model,
        'encoder_inputs' : self.encoder_inputs,
        'encoder_layers' :self.encoder_layers ,
        'decoder_inputs' :self.decoder_inputs ,
        'decoder_embedding' : self.decoder_embedding,
        'decoder_layers' : self.decoder_layers,
        'decoder_dense' : self.decoder_dense,
        'encoder_states' : self.encoder_states ,
        'params' :self.params
    }
  def compile_and_fit(self, data_dict, params):

    self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    
    summary = self.model.summary()
    plot = plot_model(self.model, show_shapes=True)
    
    train_samples = len(data_dict['train']['df']) # Total Training samples
    val_samples = len(data_dict['val']['df'])    # Total validation or test samples
    batch_size = params['batch_size']
    num_epochs = params['num_epochs']
    run_details = self.model.fit_generator(generator = data_dict['train']['batch'],
                                            steps_per_epoch = train_samples//batch_size,
                                            epochs=num_epochs,
                                            # callbacks=[BeamSearchCallBack(self.details,data_dict['val'],data_dict['tokenizer'],data_dict['out_size'],num_val=num_val_samples,beam=beam)
                                            #             #,wandb.keras.WandbCallback()
                                            #             ]
                                           validation_data = data_dict['val']['batch'],
                    validation_steps = val_samples//batch_size
                                            )

    # train_ds, val_ds = data['train'], data['val']
    # optimizer, epochs = params['optimizer'], params['num_epochs']
    
    return {
        'run_details' : run_details
    }



In [ ]:
class rnn_second() :
  def __init__(self,details) :

    self.details = details
    decoder_state_input = self.details['encoder_states']
    decoder_inputs = Input(shape=(1,))
    rep_size = self.details['params']['rep_size']
    decoder_inputs = self.details['decoder_inputs']
    x = self.details['decoder_embedding'](decoder_inputs)
    all_outputs = []
    for _ in range(self.details['params']['data_dict']['max_target_length']) :
        for layer in self.details['decoder_layers'] :
            x, *decoder_states = layer(x,initial_state=decoder_state_input)

        x = self.details['decoder_dense'](x)
        all_outputs.append(x)
        x = tf.math.argmax(x,2)  
        x = self.details['decoder_embedding'](x)
        decoder_state_input = decoder_states

    decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)
    model = Model([self.details['encoder_inputs'], decoder_inputs], decoder_outputs)
    self.model = model

  def compile_and_fit(self,data_dict,params) :
    self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    
    summary = self.model.summary()
    plot = plot_model(self.model, show_shapes=True)
    
    train_samples = len(data_dict['train']['df']) # Total Training samples
    val_samples = len(data_dict['val']['df'])    # Total validation or test samples
    batch_size = params['batch_size']
    num_epochs = params['num_epochs2']
    run_details = self.model.fit_generator(generator = data_dict['train']['batch_greedy'],
                    steps_per_epoch = train_samples//batch_size,
                    epochs=num_epochs,
                    validation_data = data_dict['val']['batch_greedy'],
                    validation_steps = val_samples//batch_size)

    # train_ds, val_ds = data['train'], data['val']
    # optimizer, epochs = params['optimizer'], params['num_epochs']
    
    return {
        'run_details' : run_details
    }

  def evaluate(self,data_dict) :
    test_gen = data_dict['test']['batch_greedy']
    test_samples = len(data_dict['test']['df'])
    batch_size=32
    acc = 0
    for _ in range(test_samples//batch_size) :
      (a,b),c = next(test_gen)
      l1 = data_dict['tokenizer'].decode(np.argmax(c,axis=2),mode='output')
      out = rnn2.model.predict([a,b])
      out = np.argmax(out,axis=2)
      l2 = data_dict['tokenizer'].decode(out,mode='output')
      acc += np.sum(np.array(l1)==np.array(l2))

    wandb.log({"Val Accuracy" : acc/test_samples})


In [ ]:
class tools:
  def init_params(config,data_dict):
  
    
    # returning parameters
    params = {
        'num_encode_layers' : config.num_encode_layers,
        'num_decode_layers' : config.num_decode_layers,
        'cell_type' : config.cell_type,
        'rep_size' : config.rep_size,
        'embed_size' : config.embed_size,
        'dropout' : config.dropout,
        'num_epochs' : config.num_epochs,
        'data_dict' : data_dict,
        'batch_size' : config.batch_size
    }
    return params

In [ ]:
# sweep configuration
sweep_config = {
    'method' : 'bayes',
    'metric' : {
        'name' : 'Val_acc',
        'goal' : 'maximize'
    },
    'parameters': {
        'cell_type' : {
            'values': ['LSTM','GRU','SimpleRNN']  
        },
        'beam': {
            'values': [2,3,4,5]
        },
        'num_val_samples': {
            'values': [200]
        },
        'embed_size': {
            'values': [16,32,64]
        },
        'rep_size': {
            'values': [32,64,128,256]
        },
        'dropout': {
            'values': [0,0.2,0.4,0.5]
        },
        'batch_size': {
            'values': [32]
        },
        'num_epochs': {
            'values': [25]
        },
        'num_encode_layers': {
            'values': [1,2,3,4,5]
        },
        'num_decode_layers': {
            'values': [1,2,3,4,5]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='dakshina_v1')

Create sweep with ID: 5to80pdt
Sweep URL: https://wandb.ai/ramkamal/dakshina_v1/sweeps/5to80pdt


In [ ]:
class sweep_module:
  @staticmethod
  def train(config=None):

    with wandb.init(config):
      
      config = wandb.config
 
      #wandb.run.name = 'fil:'+str(config['num_filters_'])+'_type:'+config['type_of_filters'][0]+'_aug:'+str(config['augmentation'])[0]+'_dro:'+str(config['dropout'])[0]
      data_dict = return_data_dict(batch_size=config.batch_size)
      params = tools.init_params(config,data_dict)
      network = rnn(params)
      run_details = network.compile_and_fit(data_dict, params)
      rnn2 = rnn_second(network.details)
      run_details2 = rnn2.compile_and_fit(data_dict)
      rnn2.evaluate(data_dict)

      if os.path.isdir('/content/wandb'): 
        shutil.rmtree('/content/wandb')

In [ ]:
sweep_id = '7g0porer'

In [ ]:
# performing the sweep
wandb.agent(sweep_id, sweep_module.train)

# Run One Model separate

In [ ]:
params = {
    'num_encode_layers' : 2,
    'num_decode_layers' : 2,
    'cell_type' : 'LSTM', # SimpleRNN, GRU
    'rep_size' : 20,
    'embed_size' : 20,
    'dropout' : 0,
    'num_epochs' : 25,
    'num_epochs2' : 25,
    'data_dict' : data_dict,
    'batch_size' : 32
}
network = rnn(params)
plot_model(network.model, show_shapes=True)
network.compile_and_fit(data_dict, params)
rnn2 = rnn_second(network.details)
run_details2 = rnn2.compile_and_fit(data_dict,params)
rnn2.evaluate(data_dict)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 20)     1440        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, None, 20)     3280        embedding[0][0]                  
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
1420/1420 [==============================] - 71s 22ms/step - loss: 1.0506 - acc: 0.2753 - val_loss: 0.7985 - val_acc: 0.3639
Epoch 2/30
1420/1420 [==============================] - 27s 19ms/step - loss: 0.8149 - acc: 0.3820 - val_loss: 0.6670 - val_acc: 0.4474
Epoch 3/30
1420/1420 [==============================] - 27s 19ms/step - loss: 0.6877 - acc: 0.4587 - val_loss: 0.5861 - val_acc: 0.5116
Epoch 4/30
1420/1420 [==============================] - 26s 19ms/step - loss: 0.6106 - acc: 0.5161 - val_loss: 0.5302 - val_acc: 0.5602
Epoch 5/30
1420/1420 [==============================] - 27s 19ms/step - loss: 0.5482 - acc: 0.5649 - val_loss: 0.4795 - val_acc: 0.6001
Epoch 6/30
1420/1420 [==============================] - 26s 19ms/step - loss: 0.4972 - acc: 0.6064 - val_loss: 0.4382 - val_acc: 0.6394
Epoch 7/30
1420/1420 [==============================] - 27s 19ms/step - loss: 0.4548 - acc: 0.6430 - val_loss: 0.4034 - val_acc: 0.6698
Epoch 8/30
1420/1420 [==========================

In [ ]:
!pip install editdistance

In [ ]:
test_gen = data_dict['test']['batch_greedy']
test_samples = len(data_dict['test']['df'])
batch_size=32
acc = 0
for _ in range(test_samples//batch_size) :
  (a,b),c = next(test_gen)
  l1 = data_dict['tokenizer'].decode(np.argmax(c,axis=2),mode='output')
  out = rnn2.model.predict([a,b])
  out = np.argmax(out,axis=2)
  l2 = data_dict['tokenizer'].decode(out,mode='output')
  acc += np.sum(np.array(l1)==np.array(l2))

print("Val Accuracy : ",acc/test_samples)

['andiranchi', 'ake', 'irangula', 'ndha', 'ulakhatee', 'alal', 'urvajanche', 'urunath', 'amjhunga', 'maging', 'tutya', 'stitvachi', 'nudit', 'addhtipeksha', 'aam', 'uloos', 'eneth', 'eevandan', 'ndreas', 'ugnata', 'alvon', 'stitvatil', 'anchyabaddal', 'otipeksha', 'umak', 'haktte', 'ubhati', 'hhavi', 'ang', 'azipur', 'illi', 'hodi']
['andyyaachi', 'ake', 'irnngda', 'ndha', 'ulakatii', 'alalt', 'urvajaache', 'urunach', 'amaaaaagt', 'minangt', 'tutya', 'sttyaache', 'nduit', 'adhititaassh', 'amttt', 'ulustt', 'anich', 'ivanaaa', 'ndyiiist', 'uganta', 'alvoo', 'stttaaaii', 'anthaaaaaadd', 'otipissha', 'umak', 'hakttt', 'ubhate', 'hhali', 'ang', 'ajiprrt', 'hlll', 'hodit']
['rut', 'eat', 'abin', 'icharakon', 'alavarun', 'rantamadhye', 'artha', 'arnayasathi', 'puri', 'hachakhach', 'arabola', 'ausa', 'ehsana', 'olanyatun', 'arlata', 'ardarala', 'uniti', 'arabai', 'emo', 'nti', 'agh', 'harge', 'yasanmuktee', 'anti', 'hambhar', 'haginee', 'anshil', 'uryadev', 'eeta', 'unaf', 'eg', 'iya']
['ret'

In [ ]:
l1 = ['andiranchi', 'ake', 'irangula', 'ndha', 'ulakhatee', 'alal', 'urvajanche', 'urunath', 'amjhunga', 'maging', 'tutya', 'stitvachi', 'nudit', 'addhtipeksha', 'aam', 'uloos', 'eneth', 'eevandan', 'ndreas', 'ugnata', 'alvon', 'stitvatil', 'anchyabaddal', 'otipeksha', 'umak', 'haktte', 'ubhati', 'hhavi', 'ang', 'azipur', 'illi', 'hodi']
l2 = ['andyyaachi', 'ake', 'irnngda', 'ndha', 'ulakatii', 'alalt', 'urvajaache', 'urunach', 'amaaaaagt', 'minangt', 'tutya', 'sttyaache', 'nduit', 'adhititaassh', 'amttt', 'ulustt', 'anich', 'ivanaaa', 'ndyiiist', 'uganta', 'alvoo', 'stttaaaii', 'anthaaaaaadd', 'otipissha', 'umak', 'hakttt', 'ubhate', 'hhali', 'ang', 'ajiprrt', 'hlll', 'hodit']
np.sum(np.array(l1)==np.array(l2))

5

In [ ]:
from keras.layers import Lambda
from keras import backend as K
num_encoder_tokens = data_dict['in_size']
latent_dim = 256
num_decoder_tokens = data_dict['out_size']
batch_size=32
epochs=20
val_samples = 4981
train_samples = 45444

# The first part is unchanged
encoder_inputs1 = Input(shape=(None,))
encoder_inputs=Embedding(data_dict['in_size'], 64,mask_zero=True)(encoder_inputs1)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
states = [state_h, state_c]

# Set up the decoder, which will only process one timestep at a time.
decoder_inputs1 = Input(shape=(1,))
decoder_embedding =  Embedding(data_dict['out_size'], 64,mask_zero=True)
decoder_inputs = decoder_embedding(decoder_inputs1)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

all_outputs = []
inputs = decoder_inputs
for _ in range(23):
    # Run the decoder on one timestep
    outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
    outputs = decoder_dense(outputs)
    # Store the current prediction (we will concatenate all predictions later)
    all_outputs.append(outputs)
    # Reinject the outputs as inputs for the next loop iteration
    # as well as update the states
    outputs1 = tf.math.argmax(outputs,2)
    inputs = decoder_embedding(outputs1)
    states = [state_h, state_c]

# Concatenate all predictions
decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

# Define and compile model as previously
model = Model([encoder_inputs1, decoder_inputs1], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# Prepare decoder input data that just contains the start character
# Note that we could have made it a constant hard-coded in the model

# # Train model as previously
# model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)

model.fit_generator(generator = data_dict['train']['batch_greedy'],
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = data_dict['val']['batch_greedy'],
                    validation_steps = val_samples//batch_size)

In [ ]:
generator=data_dict['test']['batch_greedy']
(a,b),c = next(generator)
print(a,b,np.argmax(c,axis=2))
out = model.predict([a,b])
out = np.argmax(out,axis=2)
print(out)

[[26. 40. 12. 41. 20. 15.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [33. 13. 18. 30. 21.  6. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [35. 16. 26. 16. 26.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [11. 20. 10. 20.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [28. 12. 49. 11.  9. 37. 21.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [35. 20.  6.  9. 37. 18. 10. 20. 15.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 7. 12. 13. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [37.  5. 17. 18. 13. 20.  5. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [37. 18. 45. 11. 15.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [41. 24. 15. 20.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [37.  9. 28. 29. 10. 20.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [11. 16. 41. 11. 24.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [35. 18. 13. 14. 20. 37. 15. 20. 31. 13.  0.  0.  0.  0.  0.  0.  0.]
 [13. 20. 26. 18. 23. 20. 28. 38. 21. 11.  0.  0.  0.  0.  0.  0.  0.]
 [29. 

In [ ]:
plot_model(model,to_file='model.png', show_shapes=True)

In [ ]:
encoder_inputs = Input(shape=(None,))
x = Embedding(data_dict['in_size'], 64,mask_zero=True)(encoder_inputs)
x = LSTM(units=256,return_sequences=True)(x)
x, *encoder_states = LSTM(units=256,
                           return_state=True)(x)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

decoder_embedding =  Embedding(data_dict['out_size'], 64,mask_zero=True)
x = decoder_embedding(decoder_inputs)

decoder_LSTM = LSTM(units=256, return_sequences=True, return_state=True)
x, *decoder_states = decoder_LSTM(x, initial_state=encoder_states)

decoder_dense = Dense(units=data_dict['out_size'], activation='softmax')
decoder_outputs = decoder_dense(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]
# Get the embeddings of the decoder sequence
dec_emb2 = decoder_embedding(decoder_inputs)
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs_2, *decoder_states_2 = decoder_LSTM(dec_emb2
                                                    ,initial_state=decoder_state_input
                                                    )
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs_2 = decoder_dense(decoder_outputs_2)
# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs_2] + decoder_states_2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = 1
# Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    chars = [1]
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
# Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 2 :
          stop_condition = True
        chars.append(sampled_token_index)
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
# Update states
        states_value = [h, c]
    return tk.decode([chars],'output')[0]

In [ ]:
plot_model(decoder_model,show_shapes=True)

In [ ]:
test_gen = generate_batch(X_test, Y_test, batch_size = 1)
k=-1

In [ ]:
x1 = tk.encode(['टोप्पर​'])
x1

In [ ]:
decode_sequence(x1)

In [ ]:
lex['test'].input.tolist()[k]

In [ ]:
k += 1
(input_seq, actual_output), _ = next(test_gen)
# print(input_seq)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', tk.decode([X_test[k]])[0] )
print('Actual Target Translation:', tk.decode([Y_test[k]],mode='output')[0])
print('Predicted Target Translation:', decoded_sentence)

# Romanized

In [ ]:
ta_rom = dict()
ta_rom['rejoined'] = pd.read_csv('/content/dakshina_dataset_v1.0/ta/romanized/ta.romanized.rejoined.tsv', sep='\t', header=None, error_bad_lines=False)
ta_rom['rejoined_aligned_cased'] = pd.read_csv('/content/dakshina_dataset_v1.0/ta/romanized/ta.romanized.rejoined.aligned.cased_nopunct.tsv', sep='\t', header=None, error_bad_lines=False) 
ta_rom['rejoined_aligned'] = pd.read_csv('/content/dakshina_dataset_v1.0/ta/romanized/ta.romanized.rejoined.aligned.tsv', sep='\t', header=None, error_bad_lines=False)
ta_rom['split'] = pd.read_csv('/content/dakshina_dataset_v1.0/ta/romanized/ta.romanized.split.tsv', sep='\t', header=None, error_bad_lines=False)

In [ ]:
list(ta_rom['rejoined'].iloc[0, 0])[:10]

In [ ]:
ta_rom['rejoined_aligned_cased']

In [ ]:
ta_rom['rejoined_aligned']

In [ ]:
ta_rom['split']

In [ ]:
l1 = [1,4,2,3]
l2 = [1,4,2,5]
print(np.array(l1[1:-1])==np.array(l2[1:-1]))

[ True  True]
